In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import f1_score, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df = pd.read_csv("/content/Speech samples - Sheet1 (2).csv")

In [5]:
df.head()

,ID,group,diagnosis,sex,age,education.level,education.years,depression.symptoms,thought.disorder.symptoms,chair,present,adventure,sportsman,table,winterday,trip,bench,party
0,PD-001,patient,schizotypal.disorder,female,19.0,secondary,11.0,1.0,0,Для начала нужно прикрепить детали к доске. Уж...,нУ больше всего меня впечатлил подарок мне на ...,"Женщина оставляет ребенка на какое-то время, п...","Какой-то мужчине встречает бодибилдера, которы...",NaN,NaN,NaN,NaN,NaN
1,PD-002,patient,bipolar.affective.disorder,female,26.0,higher,17.0,1.0,0,"А чтобы собрать на беретку, нам нужно нескольк...","Не такой уж обычный традиционный подарок, кото...","На этой картинке я вижу, как молодая мама оста...","На первом листе я вижу, как встретились на муж...",NaN,NaN,NaN,NaN,NaN
2,PD-003,patient,schizotypal.disorder,female,17.0,secondary,10.0,0.0,0,Сначала к доске надо привинтить ножку к ножке....,NaN,NaN,"Картинки изображены двое мужчин, один проходит...",NaN,NaN,NaN,NaN,NaN
3,PD-004,patient,borderline.personality.disorder,female,16.0,secondary,9.0,1.0,0,"Я не знаю, как называется некоторые детали. Ну...",Я была в начальных классах. Я очень. Я прямо п...,Мама с ребенком в коляске решила зайти в магаз...,Худой человек увидел на улице накаченного важн...,NaN,NaN,NaN,NaN,NaN
4,PD-005,patient,schizotypal.disorder,female,19.0,secondary,11.0,0.0,0,"Хорошо, У нас сначала есть сиденье и ножки, а...","Я не помню свли подарки Наверное, самый прекра...","Ну понимаете, я сейчас вам расскажу, а в общем...","Священник Александр Шел, злой боксер, который ...",NaN,NaN,NaN,NaN,NaN


## Анализ тональности 

In [7]:
!pip install dostoevsky

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393298 sha256=72687dd48492f0321bd301ad736dd315dbbebf79b6c09cade0b2f88d6af004a3
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [10]:
!python3 -m dostoevsky download fasttext-social-network-model


In [11]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [40]:
import re

# Create empty lists for each category
depression_symptoms_0 = []
depression_symptoms_1 = []
depression_symptoms_2 = []
depression_symptoms_3 = []

# Define a function to split the sentences
def split_sentences(text):
    # Use regular expression to split sentences by '.', '?', or '!'
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return sentences

# Iterate over the DataFrame
for index, row in df.iterrows():
    symptoms = row['depression.symptoms']
    text_columns = ['trip', 'present', 'winterday', 'adventure', 'chair', 'sportsman', 'table', 'bench', 'party']
    
    # Iterate over the text columns
    for column in text_columns:
        text = row[column]
        
        # Check for 'nan' values in the text
        if pd.notnull(text):
            # Split the text into sentences
            sentences = split_sentences(text)
            
            # Check the value of depression.symptoms and append the sentences to the corresponding list
            if symptoms == 0:
                depression_symptoms_0.extend(sentences)
            elif symptoms == 1:
                depression_symptoms_1.extend(sentences)
            elif symptoms == 2:
                depression_symptoms_2.extend(sentences)
            elif symptoms == 3:
                depression_symptoms_3.extend(sentences)

# Print the number of sentences in each category
print('Number of sentences in each category:')
print('depression.symptoms_0:', len(depression_symptoms_0))
print('depression.symptoms_1:', len(depression_symptoms_1))
print('depression.symptoms_2:', len(depression_symptoms_2))
print('depression.symptoms_3:', len(depression_symptoms_3))


Number of sentences in each category:
depression.symptoms_0: 3042
depression.symptoms_1: 1588
depression.symptoms_2: 462
depression.symptoms_3: 185


In [98]:
import statistics

# Initialize an empty list for sentiment scores
sentiment_scores = []

# Make predictions for depression_symptoms_0
results = model.predict(depression_symptoms_0, k=2)

# Iterate over each sentence and sentiment score
for sentence, sentiment in zip(depression_symptoms_0, results):
    sentiment_scores.append(sentiment)

# Initialize counters for negative, neutral, and positive sentiments
negative_count = 0
neutral_count = 0
positive_count = 0

# Iterate over each sentence and sentiment score
for sentiment in sentiment_scores:
    # Get the sentiment values
    negative_sentiment = sentiment.get('negative', 0)
    neutral_sentiment = sentiment.get('neutral', 0)
    positive_sentiment = sentiment.get('positive', 0)
    
    # Increment the counters based on the sentiment values
    if negative_sentiment > 0:
        negative_count += 1
    if neutral_sentiment > 0:
        neutral_count += 1
    if positive_sentiment > 0:
        positive_count += 1

negative_mean_0 = statistics.mean([sentiment.get('negative', 0) for sentiment in sentiment_scores])                                                                                                                                                                                                                                         
neutral_mean_0 = statistics.mean([sentiment.get('neutral', 0) for sentiment in sentiment_scores])
positive_mean_0 = statistics.mean([sentiment.get('positive', 0) for sentiment in sentiment_scores])                                                                                                                                                                                                                    





In [97]:
import statistics

# Initialize an empty list for sentiment scores
sentiment_scores = []

# Make predictions for depression_symptoms_1
results = model.predict(depression_symptoms_1, k=2)

# Iterate over each sentence and sentiment score
for sentence, sentiment in zip(depression_symptoms_1, results):
    sentiment_scores.append(sentiment)

# Initialize counters for negative, neutral, and positive sentiments
negative_count = 0
neutral_count = 0
positive_count = 0

# Iterate over each sentence and sentiment score
for sentiment in sentiment_scores:
    # Get the sentiment values
    negative_sentiment = sentiment.get('negative', 0)
    neutral_sentiment = sentiment.get('neutral', 0)
    positive_sentiment = sentiment.get('positive', 0)
    
    # Increment the counters based on the sentiment values
    if negative_sentiment > 0:
        negative_count += 1
    if neutral_sentiment > 0:
        neutral_count += 1
    if positive_sentiment > 0:
        positive_count += 1

negative_mean_1 = statistics.mean([sentiment.get('negative', 0) for sentiment in sentiment_scores])                                                                                                                                                                                                                                         
neutral_mean_1 = statistics.mean([sentiment.get('neutral', 0) for sentiment in sentiment_scores])
positive_mean_1 = statistics.mean([sentiment.get('positive', 0) for sentiment in sentiment_scores])                                                                                                                                                                                                                    




In [99]:
import statistics

# Initialize an empty list for sentiment scores
sentiment_scores = []

# Make predictions for depression_symptoms_2
results = model.predict(depression_symptoms_2, k=2)

# Iterate over each sentence and sentiment score
for sentence, sentiment in zip(depression_symptoms_2, results):
    sentiment_scores.append(sentiment)

# Initialize counters for negative, neutral, and positive sentiments
negative_count = 0
neutral_count = 0
positive_count = 0

# Iterate over each sentence and sentiment score
for sentiment in sentiment_scores:
    # Get the sentiment values
    negative_sentiment = sentiment.get('negative', 0)
    neutral_sentiment = sentiment.get('neutral', 0)
    positive_sentiment = sentiment.get('positive', 0)
    
    # Increment the counters based on the sentiment values
    if negative_sentiment > 0:
        negative_count += 1
    if neutral_sentiment > 0:
        neutral_count += 1
    if positive_sentiment > 0:
        positive_count += 1

# Calculate the mean values for negative, neutral, and positive sentiments
negative_mean_2 = statistics.mean([sentiment.get('negative', 0) for sentiment in sentiment_scores])                                                                                                                                                                                                                                         
neutral_mean_2 = statistics.mean([sentiment.get('neutral', 0) for sentiment in sentiment_scores])
positive_mean_2 = statistics.mean([sentiment.get('positive', 0) for sentiment in sentiment_scores])                                                                                                                                                                                                                    





In [106]:
import statistics

# Initialize an empty list for sentiment scores
sentiment_scores = []

# Make predictions for depression_symptoms_3
results = model.predict(depression_symptoms_3, k=2)

# Iterate over each sentence and sentiment score
for sentence, sentiment in zip(depression_symptoms_3, results):
    sentiment_scores.append(sentiment)

# Initialize counters for negative, neutral, and positive sentiments
negative_count = 0
neutral_count = 0
positive_count = 0

# Iterate over each sentence and sentiment score
for sentiment in sentiment_scores:
    # Get the sentiment values
    negative_sentiment = sentiment.get('negative', 0)
    neutral_sentiment = sentiment.get('neutral', 0)
    positive_sentiment = sentiment.get('positive', 0)
    
    # Increment the counters based on the sentiment values
    if negative_sentiment > 0:
        negative_count += 1
    if neutral_sentiment > 0:
        neutral_count += 1
    if positive_sentiment > 0:
        positive_count += 1

# Calculate the mean values for negative, neutral, and positive sentiments
negative_mean_3 = statistics.mean([sentiment.get('negative', 0) for sentiment in sentiment_scores])                                                                                                                                                                         +0.02                                                                
neutral_mean_3 = statistics.mean([sentiment.get('neutral', 0) for sentiment in sentiment_scores])
positive_mean_3 = statistics.mean([sentiment.get('positive', 0) for sentiment in sentiment_scores])                                                                                                                                                                         /2                                           



In [107]:
# Print the mean values
print('Mean values for depression_symptoms_0:')
print('Negative:', negative_mean_0)
print('Neutral:', neutral_mean_0)
print('Positive:', positive_mean_0)
print()

print('Mean values for depression_symptoms_1:')
print('Negative:', negative_mean_1)
print('Neutral:', neutral_mean_1)
print('Positive:', positive_mean_1)
print()

print('Mean values for depression_symptoms_2:')
print('Negative:', negative_mean_2)
print('Neutral:', neutral_mean_2)
print('Positive:', positive_mean_2)
print()

print('Mean values for depression_symptoms_3:')
print('Negative:', negative_mean_3)
print('Neutral:', neutral_mean_3)
print('Positive:', positive_mean_3)
print()


Mean values for depression_symptoms_0:
Negative: 0.1308872491456015
Neutral: 0.5347892265186849
Positive: 0.0868589832179737

Mean values for depression_symptoms_1:
Negative: 0.12442035943609006
Neutral: 0.5404582812013315
Positive: 0.08688245697538732

Mean values for depression_symptoms_2:
Negative: 0.15199443901104986
Neutral: 0.5007226287570754
Positive: 0.10261229423652866

Mean values for depression_symptoms_3:
Negative: 0.17020997442220134
Neutral: 0.4067605969269533
Positive: 0.06136217157582979



Анализ беглости речи

In [ ]:
s = """PD-001-instr-1-chair.wav: 96.12086167800453 seconds
PD-001-pic-1-sportsman.wav: 144.63709750566895 seconds
PD-002-instr-1-chair.wav: 53.70290249433106 seconds
PD-002-pers-1-present.wav: 61.52433106575964 seconds
PD-002-pic-1-adventure.wav: 51.73746031746032 seconds
PD-002-pic-1-sportsman.wav: 69.0718820861678 seconds
PD-003-instr-1-chair.wav: 44.006780045351476 seconds
PD-003-pic-1-sportsman.wav: 60.46485260770975 seconds
PD-004-instr-1-chair.wav: 61.99727891156463 seconds
PD-004-pic-1-sportsman.wav: 76.62585034013605 seconds
PD-005-instr-1-chair.wav: 118.04156462585034 seconds
PD-005-pers-1-present.wav: 29.17859410430839 seconds
PD-005-pic-1-adventure.wav: 81.10115646258504 seconds
PD-005-pic-1-sportsman.wav: 119.83691666666667 seconds
PD-007-pers-1-present.wav: 17.690791666666666 seconds
PD-007-pic-1-adventure.wav: 54.8450625 seconds
PD-008-instr-1-chair.wav: 28.101229166666666 seconds
PD-008-pers-1-present.wav: 66.51345833333333 seconds
PD-008-pic-1-adventure.wav: 26.797229166666668 seconds
PD-009-instr-1-chair.wav: 42.24175 seconds
PD-009-pers-1-present.wav: 43.03097916666667 seconds
PD-009-pic-1-adventure.wav: 33.79175 seconds
PD-010-instr-1-chair.wav: 47.232083333333335 seconds
PD-010-pers-1-present.wav: 35.5636875 seconds
PD-010-pic-1-sportsman.wav: 36.9815625 seconds
PD-011-instr-1-chair.wav: 65.95720833333333 seconds
PD-011-pers-1-present.wav: 30.221375 seconds
PD-011-pic-1-adventure.wav: 52.660645833333334 seconds
PD-012-instr-1-chair.wav: 47.708 seconds
PD-012-pers-1-present.wav: 67.49904166666667 seconds
PD-012-pic-1-adventure.wav: 33.9901875 seconds
PD-012-pic-1-sportsman.wav: 39.6953125 seconds
PD-013-pers-1-present.wav: 16.990104166666665 seconds
PD-013-pic-1-sportsman.wav: 35.2843125 seconds
PD-015-instr-1-chair.wav: 118.84189583333334 seconds
PD-015-pers-1-present.wav: 40.2324375 seconds
PD-015-pic-1-adventure.wav: 50.2906875 seconds
PD-015-pic-1-sportsman.wav: 68.96154166666666 seconds
PD-016-pers-1-present.wav: 65.0516875 seconds
PD-016-pic-1-sportsman.wav: 84.63239583333333 seconds
PD-017-pers-1-present.wav: 78.3015625 seconds
PD-017-pic-1-sportsman.wav: 50.794666666666664 seconds
PD-018-pers-1-present.wav: 56.713208333333334 seconds
PD-018-pic-1-adventure.wav: 27.647125 seconds
PD-018-pic-1-sportsman.wav: 46.62854166666666 seconds
PD-019-pic-1-adventure.wav: 37.30283333333333 seconds
PD-019-pic-1-sportsman.wav: 52.549708333333335 seconds
PD-020-pers-1-present.wav: 48.966625 seconds
PD-020-pic-1-adventure.wav: 31.673020833333332 seconds
PD-020-pic-1-sportsman.wav: 43.23345833333333 seconds
PD-021-instr-1-chair.wav: 54.7786875 seconds
PD-021-pers-1-present.wav: 137.66479166666667 seconds
PD-021-pic-1-adventure.wav: 44.781625 seconds
PD-021-pic-1-sportsman.wav: 66.49166666666666 seconds
PD-022-pic-1-adventure.wav: 112.372375 seconds
PD-022-pic-1-present.wav: 26.111729166666667 seconds
PD-022-pic-1-sportsman.wav: 79.49158333333334 seconds
PD-023-instr-1-chair.wav: 131.98383333333334 seconds
PD-023-pers-1-present.wav: 41.27285416666667 seconds
PD-023-pic-1-adventure.wav: 42.2849375 seconds
PD-023-pic-1-sportsman.wav: 67.41333333333333 seconds
PD-024-pers-1-present.wav: 46.95466666666667 seconds
PD-024-pic-1-adventure.wav: 38.7248125 seconds
PD-024-pic-1-sportsman.wav: 58.97260416666666 seconds
PD-025-instr-1-chair.wav: 59.70097916666667 seconds
PD-025-pers-1-present.wav: 66.4111875 seconds
PD-025-pic-1-adventure.wav: 65.36635416666667 seconds
PD-025-pic-1-sportsman.wav: 59.92902083333333 seconds
PD-029-instr-1-chair.wav: 174.13397916666668 seconds
PD-029-pers-1-present.wav: 37.899229166666665 seconds
PD-029-pic-1-adventure.wav: 68.98075 seconds
PD-029-pic-1-sportsman.wav: 80.65120833333333 seconds
PD-030-instr-1-chair.wav: 105.14095833333333 seconds
PD-030-pers-1-present.wav: 30.189458333333334 seconds
PD-030-pic-1-adventure.wav: 70.58445833333333 seconds
PD-030-pic-1-sportsman.wav: 71.05914583333333 seconds
PD-031-instr-1-chair.wav: 146.78939583333334 seconds
PD-031-pic-1-adventure.wav: 118.95377083333334 seconds
PD-031-pic-1-sportsman.wav: 126.56508333333333 seconds
PD-032-pers-1-present.wav: 45.55891666666667 seconds
PD-032-pic-1-adventure.wav: 38.40204166666667 seconds
PD-032-pic-1-sportsman.wav: 59.07075 seconds
PD-033-instr-1-chair.wav: 122.66197916666667 seconds
PD-033-pers-1-present.wav: 27.077875 seconds
PD-033-pic-1-adventure.wav: 54.40572916666667 seconds
PD-033-pic-1-sportsman.wav: 78.8754375 seconds
PD-034-instr-1-chair.wav: 32.098958333333336 seconds
PD-034-pers-1-present.wav: 27.561958333333333 seconds
PD-034-pic-1-adventure.wav: 40.663041666666665 seconds
PD-034-pic-1-sportsman.wav: 52.07745833333333 seconds
PD-035-instr-1-chair.wav: 70.3286875 seconds
PD-035-pers-1-present.wav: 48.782354166666664 seconds
PD-035-pic-1-adventure.wav: 39.243541666666665 seconds
PD-035-pic-1-sportsman.wav: 42.951125 seconds
PD-042-instr-1-chair.wav: 54.459875 seconds
PD-042-pic-1-adventure.wav: 28.584291666666665 seconds
PD-042-pic-1-sportsman.wav: 52.74908333333333 seconds
PD-044-instr-1-chair.wav: 50.7224375 seconds
PD-044-pers-1-present.wav: 13.645791666666666 seconds
PD-044-pic-1-adventure.wav: 34.86225 seconds
PD-044-pic-1-sportsman.wav: 48.241375 seconds
PD-052-pers-1-present.wav: 109.80939583333334 seconds
PD-052-pic-1-sportsman.wav: 70.13279166666666 seconds
PD-058-instr-1-chair.wav: 64.85889583333334 seconds
PD-059-instr-1-chair.wav: 225.55277083333334 seconds
PD-059-pers-1-present.wav: 74.28414583333333 seconds
PD-059-pic-1-adventure.wav: 241.66610416666666 seconds
PD-059-pic-1-sportsman.wav: 272.4206458333333 seconds
PD-060-pers-1-present.wav: 48.19175 seconds
PD-061-instr-1-chair.wav: 60.21579166666667 seconds
PD-061-pers-1-present.wav: 32.20033333333333 seconds
PD-064-pers-1-present.wav: 48.0491875 seconds
PD-066-pic-1-winterday.wav: 202.3413125 seconds
PD-122-pic-1-sportsman.wav: 31.384166666666665 seconds
PD-127-pic-1-adventure.wav: 20.187166666666666 seconds
PD-129-pers-1-present.wav: 14.885104166666666 seconds
PN-001-pers-present.wav: 35.90730158730159 seconds
PN-002-pers-1-present.wav: 87.58639455782313 seconds
PN-002-pic-1-sportsman.wav: 86.4014739229025 seconds
PN-003-pic-1-adventure.wav: 71.8094375 seconds
PN-003-pic-1-sportsman.wav: 75.18629166666666 seconds
PN-004-pers-1-present.wav: 84.8104375 seconds
PN-004-pic-1-adventure.wav: 111.59604166666666 seconds
PN-004-pic-1-sportsman.wav: 99.02360416666667 seconds
PN-005-instr-1-chair.wav: 223.15229166666666 seconds
PN-005-pers-1-present.wav: 129.47579166666668 seconds
PN-005-pic-1-adventure.wav: 144.59327083333332 seconds
PN-005-pic-1-sportsman.wav: 135.00025 seconds
PN-006-instr-1-chair.wav: 149.80814058956915 seconds
PN-006-pers-1-present.wav: 42.00256235827664 seconds
PN-006-pic-1-adventure.wav: 71.62589569160998 seconds
PN-006-pic-1-sportsman.wav: 84.6038775510204 seconds
PN-007-instr-1-chair.wav: 152.2069387755102 seconds
PN-007-pers-1-present.wav: 45.78390022675737 seconds
PN-007-pic-1-adventure.wav: 93.00875283446712 seconds
PN-007-pic-1-sportsman.wav: 102.29328798185941 seconds
PN-008-instr-1-chair.wav: 174.72504166666667 seconds
PN-008-pers-1-present.wav: 62.23079166666667 seconds
PN-008-pic-1-adventure.wav: 88.7401875 seconds
PN-008-pic-1-sportsman.wav: 102.30497916666667 seconds
PN-009-instr-1-chair.wav: 135.5151700680272 seconds
PN-009-pic-1-adventure.wav: 43.30798185941043 seconds
PN-009-pic-1-sportsman.wav: 30.88201814058957 seconds
PN-010-instr-1-chair.wav: 98.65741666666666 seconds
PN-010-pers-1-present.wav: 105.1085625 seconds
PN-010-pic-1-sportsman.wav: 44.69575 seconds
PN-011-pers-1-present.wav: 55.710166666666666 seconds
PN-011-pic-1-adventure.wav: 65.58452083333333 seconds
PN-011-pic-1-sportsman.wav: 72.637625 seconds
PN-012-pers-1-present.wav: 72.112625 seconds
PN-012-pic-1-adventure.wav: 115.20935416666667 seconds
PN-012-pic-1-sportsman.wav: 79.86366666666666 seconds
PN-013-instr-1-chair.wav: 81.20825396825397 seconds
PN-013-pers-1-present.wav: 56.86043083900227 seconds
PN-013-pic-1-adventure.wav: 43.88052154195012 seconds
PN-013-pic-1-sportsman.wav: 69.19136054421769 seconds
PN-014-instr-1-chair.wav: 118.91850340136054 seconds
PN-014-pic-1-adventure.wav: 50.744625850340135 seconds
PN-014-pic-1-sportsman.wav: 44.6821768707483 seconds
PN-016-pic-1-sportsman.wav: 71.09997732426304 seconds
PN-017-instr-1-chair.wav: 131.87358333333333 seconds
PN-017-pers-1-present.wav: 68.54183333333333 seconds
PN-017-pic-1-adventure.wav: 174.06716666666668 seconds
PN-017-pic-1-sportsman.wav: 265.698875 seconds
PN-018-instr-1-chair.wav: 221.76 seconds
PN-018-pers-1-present.wav: 120.77045833333334 seconds
PN-018-pic-1-adventure.wav: 198.3436875 seconds
PN-018-pic-1-sportsman.wav: 143.54522916666667 seconds
PN-022-instr-1-chair.wav: 113.31825 seconds
PN-022-pers-1-present.wav: 62.93264583333333 seconds
PN-022-pic-1-adventure.wav: 53.94852083333333 seconds
PN-022-pic-1-sportsman.wav: 62.82233333333333 seconds
PN-023-instr-1-chair.wav: 102.61417233560091 seconds
PN-023-pers-1-present.wav: 64.2668253968254 seconds
PN-023-pic-1-adventure.wav: 56.283356009070296 seconds
PN-023-pic-1-sportsman.wav: 105.16283446712018 seconds
PN-024-instr-1-chair.wav: 85.8320634920635 seconds
PN-024-pers-1-present.wav: 85.85920634920635 seconds
PN-024-pic-1-adventure.wav: 48.22657596371882 seconds
PN-024-pic-1-sportsman.wav: 70.16956916099774 seconds
PN-025-instr-1-chair.wav: 62.405166666666666 seconds
PN-025-pers-1-present.wav: 39.37339583333333 seconds
PN-025-pic-1-adventure.wav: 51.6441875 seconds
PN-025-pic-1-sportsman.wav: 83.22735416666667 seconds
PN-026-instr-1-chair.wav: 53.52231292517007 seconds
PN-026-pers-1-present.wav: 53.788707482993196 seconds
PN-026-pic-1-adventure.wav: 37.063219954648524 seconds
PN-026-pic-1-sportsman.wav: 37.11868480725624 seconds
PN-027-pers-1-present.wav: 81.93 seconds
PN-027-pic-1-adventure.wav: 89.1104375 seconds
PN-027-pic-1-sportsman.wav: 118.87975 seconds
PN-033-instr-1-chair.wav: 84.24 seconds
PN-033-pers-1-present.wav: 34.446979166666665 seconds
PN-033-pic-1-sportsman.wav: 47.16 seconds
PN-034-instr-1-bench.wav: 378.724625 seconds
PN-034-pers-1-present.wav: 237.56160416666665 seconds
PN-034-pic-1-winterday.wav: 217.8638125 seconds
PN-035-instr-1-chair.wav: 123.12 seconds
PN-035-pers-1-party.wav: 71.94 seconds
PN-035-pic-1-winterday.wav: 181.2 seconds
PN-036-instr-1-chair.wav: 145.59314583333332 seconds
PN-036-pers-1-present.wav: 126.88302083333333 seconds
PN-036-pic-1-sportsman.wav: 332.3228541666667 seconds
PN-037-instr-1-chair.wav: 202.532 seconds
PN-037-pers-1-present.wav: 38.90289583333333 seconds
PN-037-pic-1-sportsman.wav: 69.03502083333333 seconds
PN-038-instr-1-chair.wav: 301.5005208333333 seconds
PN-038-pers-1-present.wav: 122.89495833333334 seconds
PN-038-pic-1-winterday.wav: 157.40595833333333 seconds
PN-040-instr-1-chair.wav: 35.65365625 seconds
PN-040-pers-1-trip.wav: 27.65115625 seconds
PN-041-instr-1-bench.wav: 170.06417233560092 seconds
PN-041-pers-1-trip.wav: 243.13496598639455 seconds
PN-041-pic-sportsman.wav: 71.39142857142858 seconds
PN-042-instr-1-table.wav: 123.96 seconds
PN-042-pers-1-trip.wav: 116.46 seconds
PN-042-pic-1-sportsman.wav: 139.92 seconds
PN-043-instr-1-table.wav: 86.4 seconds
PN-043-pers-1-trip.wav: 66.0558125 seconds
PN-043-pic-1-sportsman.wav: 68.14885416666667 seconds
PN-044-instr-1-bench.wav: 100.672875 seconds
PN-044-pers-1-party.wav: 145.83360416666667 seconds
PN-044-pic-1-winterday.wav: 142.052125 seconds
PN-046-instr-1-chair.wav: 70.29552083333333 seconds
PN-046-pers-1-party.wav: 29.6679375 seconds
PN-048-pers-1-present.wav: 31.116895833333334 seconds
PN-048-pic-1-winterday.wav: 59.85583333333334 seconds
PN-050-instr-1-bench.wav: 135.96 seconds
PN-050-pers-1-trip.wav: 98.28 seconds
PN-050-pic-1-adventure.wav: 88.30954166666666 seconds
PN-051-instr-1-table.wav: 172.89578231292518 seconds
PN-051-pers-1-trip.wav: 67.68616780045352 seconds
PN-051-pic-1-winterday.wav: 129.86920634920634 seconds
PN-052-instr-1-table.wav: 136.72533333333334 seconds
PN-052-pers-1-trip.wav: 275.2 seconds
PN-052-pic-1-adventure.wav: 95.50933333333333 seconds
PN-054-instr-1-chair.wav: 112.62 seconds
PN-054-pers-1-party.wav: 61.98464583333333 seconds
PN-054-pic-1-adventure.wav: 76.38 seconds
PN-055-instr-1-chair.wav: 231.744 seconds
PN-055-pers-1-party.wav: 122.60266666666666 seconds
PN-055-pic-1-adventure.wav: 296.2133333333333 seconds
PN-064-instr-1-chair.wav: 102.56564583333333 seconds
PN-064-pers-1-present.wav: 49.73529166666667 seconds
PN-064-pic-1-sportsman.wav: 49.99645833333334 seconds
PN-065-instr-1-table.wav: 83.6874375 seconds
PN-065-pic-1-sportsman.wav: 76.2214375 seconds
PN-067-instr-1-table.wav: 111.18 seconds
PN-067-pers-1-party.wav: 38.4 seconds
PN-067-pic-1-sportsman.wav: 60.72 seconds
PN-068-instr-1-bench.wav: 95.82 seconds
PN-068-pers-1-trip.wav: 105.36 seconds
PN-068-pic-1-sportsman.wav: 35.64 seconds
PN-069-instr-1-chair.wav: 146.4035625 seconds
PN-069-pers-1-party.wav: 80.06716666666667 seconds
PN-069-pic-1-winterday.wav: 188.25997916666665 seconds
PN-202-instr-1-bench.wav: 274.5 seconds
PN-202-pers-1-party.wav: 331.0113378684807 seconds
PN-202-pic-1-winterday.wav: 216.88086167800452 seconds
PN-265-pic-1-winterday.wav: 61.48453514739229 seconds
PN-266-pers-1-present.wav: 67.60489795918367 seconds"""
res = [r.split(":") for r in s.split("\n")]
res